In [1]:
import os
AVAILABLE_GPUS = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = AVAILABLE_GPUS

import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, GPTNeoXForCausalLM

for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

NVIDIA RTX A6000


In [2]:
model_name_or_path = "OpenAssistant/pythia-12b-sft-v8-rlhf-2k-steps" #23GB

model = GPTNeoXForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side="left", legacy=False)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import pandas as pd
import json

cut_off = 2000

def load_from_jsonl(file_name: str, cut_off: int) -> list[dict]:
    def load_json_line(line: str, i: int, file_name: str):
        try:
            return json.loads(line)
        except:
            raise ValueError(f"Error in line {i+1}\n{line} of {file_name}")
    with open(file_name, "r") as f:
        data = [load_json_line(line, i, file_name) for i, line in enumerate(f) if i < cut_off]
    return data

def get_prompt(d):
    prompt = d['prompt'][0]['content']
    OASS_prompt = '<|prompter|>' + prompt + '<|endoftext|>\n<|assistant|>Here are my thoughts on the poem:\n'
    return OASS_prompt 

dataset_filename: str = "feedback_poem.jsonl"
dataset_path = os.path.join("datasets", dataset_filename)
dataset = load_from_jsonl(dataset_path, cut_off)
print("Number of samples:",len(dataset))
prompts= [get_prompt(d) for d in dataset]

Number of samples: 2000


In [4]:
output_array = []
for i in range(0, len(prompts), 5):
    print("step: " +str(i//5) + " out of: " +str(len(prompts)//5))
    model_inputs = tokenizer(prompts[i:i+5], return_tensors="pt", padding=True).to("cuda")
    generated_ids = model.generate(**model_inputs, max_length=800, pad_token_id=tokenizer.eos_token_id, do_sample=True, temperature=0.2)
    output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    output_array.append(output)

    # for i in range(5):
    #     #print(prompts[i])
    #     print(output[i].split('Here are my thoughts on the poem:')[1])
    #     print("#############################################")

step: 0 out of: 400
step: 1 out of: 400
step: 2 out of: 400
step: 3 out of: 400
step: 4 out of: 400
step: 5 out of: 400
step: 6 out of: 400
step: 7 out of: 400
step: 8 out of: 400
step: 9 out of: 400
step: 10 out of: 400
step: 11 out of: 400
step: 12 out of: 400
step: 13 out of: 400
step: 14 out of: 400
step: 15 out of: 400
step: 16 out of: 400
step: 17 out of: 400
step: 18 out of: 400
step: 19 out of: 400
step: 20 out of: 400
step: 21 out of: 400
step: 22 out of: 400
step: 23 out of: 400
step: 24 out of: 400
step: 25 out of: 400
step: 26 out of: 400
step: 27 out of: 400
step: 28 out of: 400
step: 29 out of: 400
step: 30 out of: 400
step: 31 out of: 400
step: 32 out of: 400
step: 33 out of: 400
step: 34 out of: 400
step: 35 out of: 400
step: 36 out of: 400
step: 37 out of: 400
step: 38 out of: 400
step: 39 out of: 400
step: 40 out of: 400
step: 41 out of: 400
step: 42 out of: 400
step: 43 out of: 400
step: 44 out of: 400
step: 45 out of: 400
step: 46 out of: 400
step: 47 out of: 400
st

In [5]:
# flatten output array
flat_output_array = [item for sublist in output_array for item in sublist]

# remove prompt from output
cleaned_output_array = [item.split('Here are my thoughts on the poem:\n')[1] for item in flat_output_array]

In [6]:
import datetime
now = datetime.datetime.now()

#create pandas dataframe with the array prompts as attribute prompt and the cleaned_output_array as attribute response
df = pd.DataFrame({'prompt': prompts, 'response': cleaned_output_array})

#create a new "type" column and fille it with "base" if the index is a multiple of 5, "like" if index%5 == 1, "dislike" if index%5 == 2, "wrote" if index%5 == 3, "not_wrote" if index%5 == 4
df['type'] = df.index.map(lambda x: 'base' if x%5 == 0 else 'like' if x%5 == 1 else 'dislike' if x%5 == 2 else 'wrote' if x%5 == 3 else 'not_wrote')

print(df.head())

#create a file name with current date and time
date_time = now.strftime("%m_%d_%Y_%H_%M_%S")
file_name = (model_name_or_path + "_" + date_time + "_" + dataset_filename.split(".")[0] + "_" + str(cut_off) + ".csv").replace("/", "_")
file_path = "results/" + file_name
print("file name:", file_name)

#save dataframe as csv
df.to_csv(file_path, index=False)

                                              prompt  \
0  <|prompter|>Please comment briefly on the foll...   
1  <|prompter|>Please comment briefly on the foll...   
2  <|prompter|>Please comment briefly on the foll...   
3  <|prompter|>Please comment briefly on the foll...   
4  <|prompter|>Please comment briefly on the foll...   

                                            response       type  
0  \nThis poem is a beautiful and thought-provoki...       base  
1  \nThis poem is a beautiful and thought-provoki...       like  
2  \nOverall, the poem seems to be a celebration ...    dislike  
3  \nThis poem is a beautiful and thought-provoki...      wrote  
4  \nThis poem is a beautiful and thought-provoki...  not_wrote  
file name: OpenAssistant_pythia-12b-sft-v8-rlhf-2k-steps_11_13_2023_13_01_03_feedback_poem_2000.csv
